# ⭐ !pip install & Connector

In [ ]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 32.6 MB/s eta 0:00:00


In [ ]:
db_user = "aie222"
db_password = "112233"
db_server = "clickrouting.com"
db_name = "db_bu_student_aie222"

import mysql.connector
conn = mysql.connector.connect(host=db_server, user=db_user, password=db_password, db=db_name)
cur = conn.cursor()

# ⭐ CREATE TABLE



In [ ]:
# สร้างตาราง books_clean
cur.execute("""
CREATE TABLE IF NOT EXISTS books_clean (
    book_id INT PRIMARY KEY,
    title VARCHAR(255),
    author VARCHAR(150),
    genre VARCHAR(50),
    price DECIMAL(10,2),
    stock INT
)
""")
print("✅ Table 'books_clean' created!")

# สร้างตาราง booksales_clean
cur.execute("""
CREATE TABLE IF NOT EXISTS booksales_clean (
    order_id INT PRIMARY KEY,
    customer_id INT,
    book_id INT,
    quantity INT,
    order_date DATE,
    total_price DECIMAL(10,2),
    FOREIGN KEY (book_id) REFERENCES books_clean(book_id)
)
""")
print("✅ Table 'booksales_clean' created!")
conn.commit()

✅ Table 'books_clean' created!
✅ Table 'booksales_clean' created!


# ⭐ UPLOAD FILES (.csv)



In [ ]:
from google.colab import files
uploaded = files.upload()

Saving booksales_clean.csv to booksales_clean (5).csv
Saving books_clean.csv to books_clean (5).csv


In [ ]:
import csv

# Import books_clean.csv → ตาราง books_clean
with open("books_clean.csv", newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    header = next(reader)
    print(f"📋 Books columns: {header}")

    for row in reader:
        row = [None if val == '' else val for val in row]

        cur.execute("""
            INSERT INTO books_clean (book_id, title, author, genre, price, stock)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, row)

conn.commit()
print("✅ Books data inserted!")

# Import booksales_clean.csv → ตาราง booksales_clean
with open("booksales_clean.csv", newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    header = next(reader)
    print(f"📋 Sales columns: {header}")

    for row in reader:
        row = [None if val == '' else val for val in row]

        cur.execute("""
            INSERT INTO booksales_clean (order_id, customer_id, book_id,
                                         quantity, order_date, total_price)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, row)

conn.commit()
print("✅ Sales data inserted!")

# ตรวจสอบผลลัพธ์
cur.execute("SELECT COUNT(*) FROM books_clean")
print(f"📚 Total books: {cur.fetchone()[0]}")

cur.execute("SELECT COUNT(*) FROM booksales_clean")
print(f"🛒 Total sales: {cur.fetchone()[0]}")

📋 Books columns: ['book_id', 'title', 'author', 'genre', 'price', 'stock']
✅ Books data inserted!
📋 Sales columns: ['order_id', 'customer_id', 'book_id', 'quantity', 'order_date', 'total_price']
✅ Sales data inserted!
📚 Total books: 10
🛒 Total sales: 120


In [ ]:
import pandas as pd

# ===================================
# KPI 1: Total Sales Revenue
# ===================================
print("=" * 60)
print("KPI 1: Total Sales Revenue - วัดประสิทธิภาพการขายโดยรวม")
print("=" * 60)

sql_revenue = """
SELECT
    SUM(total_price) AS total_revenue,
    COUNT(DISTINCT order_id) AS total_orders,
    COUNT(DISTINCT customer_id) AS total_customers,
    ROUND(SUM(total_price) / COUNT(DISTINCT order_id), 2) AS avg_revenue_per_order
FROM booksales_clean
"""

cur.execute(sql_revenue)
rows = cur.fetchall()
column_names = [i[0] for i in cur.description]
df_revenue = pd.DataFrame(rows, columns=column_names)
print(df_revenue)
print("\n")

# ===================================
# KPI 2: Sales by Category (Genre)
# ===================================
print("=" * 60)
print("KPI 2: Sales by Category - วัดประสิทธิภาพของแต่ละหมวดสินค้า")
print("=" * 60)

sql_category = """
SELECT
    b.genre,
    COUNT(DISTINCT bs.order_id) AS total_orders,
    SUM(bs.quantity) AS total_books_sold,
    SUM(bs.total_price) AS total_revenue,
    ROUND(AVG(bs.total_price), 2) AS avg_order_value,
    ROUND(SUM(bs.total_price) * 100.0 /
          (SELECT SUM(total_price) FROM booksales_clean), 2) AS revenue_percentage
FROM booksales_clean bs
JOIN books_clean b ON bs.book_id = b.book_id
GROUP BY b.genre
ORDER BY total_revenue DESC
"""

cur.execute(sql_category)
rows = cur.fetchall()
column_names = [i[0] for i in cur.description]
df_category = pd.DataFrame(rows, columns=column_names)
print(df_category)
print("\n")

# ===================================
# KPI 3: Average Order Value (AOV)
# ===================================
print("=" * 60)
print("KPI 3: Average Order Value - วัดพฤติกรรมการใช้จ่ายของลูกค้า")
print("=" * 60)

sql_aov = """
SELECT
    customer_id,
    COUNT(order_id) AS total_orders,
    SUM(total_price) AS total_spent,
    ROUND(AVG(total_price), 2) AS avg_order_value,
    SUM(quantity) AS total_books_purchased,
    ROUND(SUM(total_price) / SUM(quantity), 2) AS avg_price_per_book
FROM booksales_clean
GROUP BY customer_id
ORDER BY total_spent DESC
LIMIT 10
"""

cur.execute(sql_aov)
rows = cur.fetchall()
column_names = [i[0] for i in cur.description]
df_aov = pd.DataFrame(rows, columns=column_names)
print(df_aov)
print("\n")

# ===================================
# สรุป KPI Overview
# ===================================
print("=" * 60)
print("📊 KPI SUMMARY")
print("=" * 60)
print(f"💰 Total Revenue: ${df_revenue['total_revenue'][0]:,.2f}")
print(f"📦 Total Orders: {df_revenue['total_orders'][0]}")
print(f"👥 Total Customers: {df_revenue['total_customers'][0]}")
print(f"💵 Avg Revenue per Order: ${df_revenue['avg_revenue_per_order'][0]:,.2f}")
print(f"📚 Top Selling Genre: {df_category.iloc[0]['genre']} (${df_category.iloc[0]['total_revenue']:,.2f})")
print("=" * 60)

KPI 1: Total Sales Revenue - วัดประสิทธิภาพการขายโดยรวม
  total_revenue  total_orders  total_customers avg_revenue_per_order
0      45442.00           120              106                378.68


KPI 2: Sales by Category - วัดประสิทธิภาพของแต่ละหมวดสินค้า
       genre  total_orders total_books_sold total_revenue avg_order_value  \
0      Comic            68              199      18962.00          278.85   
1      Novel            27               40      12430.00          460.37   
2  Education            15               22       9065.00          604.33   
3   Business            10               13       4985.00          498.50   

  revenue_percentage  
0              41.73  
1              27.35  
2              19.95  
3              10.97  


KPI 3: Average Order Value - วัดพฤติกรรมการใช้จ่ายของลูกค้า
   customer_id  total_orders total_spent avg_order_value  \
0          NaN             4     1974.00          493.50   
1        109.0             2     1190.00          595.00   
2